# Convolutional network

Adapted from [Pytorch tutorial for Deep Learning researchers](https://github.com/yunjey/pytorch-tutorial) (Yunvey Choi, 2018).

Used as part of Deep Learning, Gilles Louppe, 2018.

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# Hyper-parameters

In [2]:
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# Data

In [3]:
# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Model

In [4]:
# Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = CNN()

# Loss and optimizer

In [5]:
# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                  % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss))

Epoch: [1/5], Step: [100/600], Loss: 1.9691
Epoch: [1/5], Step: [200/600], Loss: 1.5113
Epoch: [1/5], Step: [300/600], Loss: 1.2245
Epoch: [1/5], Step: [400/600], Loss: 0.9624
Epoch: [1/5], Step: [500/600], Loss: 0.8503
Epoch: [1/5], Step: [600/600], Loss: 0.6662
Epoch: [2/5], Step: [100/600], Loss: 0.5966
Epoch: [2/5], Step: [200/600], Loss: 0.4791
Epoch: [2/5], Step: [300/600], Loss: 0.4781
Epoch: [2/5], Step: [400/600], Loss: 0.4480
Epoch: [2/5], Step: [500/600], Loss: 0.4640
Epoch: [2/5], Step: [600/600], Loss: 0.4122
Epoch: [3/5], Step: [100/600], Loss: 0.4079
Epoch: [3/5], Step: [200/600], Loss: 0.2862
Epoch: [3/5], Step: [300/600], Loss: 0.3484
Epoch: [3/5], Step: [400/600], Loss: 0.3652
Epoch: [3/5], Step: [500/600], Loss: 0.2772
Epoch: [3/5], Step: [600/600], Loss: 0.2656
Epoch: [4/5], Step: [100/600], Loss: 0.3021
Epoch: [4/5], Step: [200/600], Loss: 0.2957
Epoch: [4/5], Step: [300/600], Loss: 0.2438
Epoch: [4/5], Step: [400/600], Loss: 0.2115
Epoch: [4/5], Step: [500/600], L

# Test the model

In [6]:
# Test the Model
model.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0

for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the model on the 10000 test images: 95 %


<div class="alert alert-success">
<b>EXERCISE</b>:

Write two convolutional networks, designed as:
<ul>
    <li>INPUT→[CONV→RELU→POOL]*2→FC→RELU→FC</li>
    <li>INPUT→[[CONV→RELU]*2→POOL]*3→[FC→RELU]*2→FC</li>
</ul>

</div>

<div class="alert alert-success">
<b>EXERCISE</b>:

Count the number of parameters of these networks

</div>

<div class="alert alert-success">
<b>EXERCISE</b>:

Which one work best on MNIST?

</div>

<div class="alert alert-success">
<b>EXERCISE</b>:

Evaluate the effect of the number of convolutional kernels on the final model performance.

</div>

<div class="alert alert-success">
<b>EXERCISE</b>:

Train a convolutional neural network on <code>dsets.CIFAR10</code>.

</div>

# Using a pretrained model

In [7]:
# Download and load pretrained resnet.
import torchvision

In [8]:
resnet = torchvision.models.resnet18(pretrained=True)

In [11]:
# If you want to finetune only top layer of the model.
for param in resnet.parameters():
    param.requires_grad = True
    
# Replace top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 10)  

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(count_parameters(resnet))

11181642


In [13]:
# For test.
images = torch.randn(10, 3, 224, 224)
outputs = resnet(images)
print(outputs)  

tensor([[-0.4083, -0.0396, -0.4183, -0.7887,  0.2547, -0.0193, -0.9379,  0.4512,
         -0.7356, -1.3701],
        [-0.4878, -0.2544, -1.0924, -1.0049,  0.2767, -0.4340, -0.0540, -0.3392,
         -0.6985, -1.5758],
        [-0.6354,  0.3880,  0.0790, -1.5181,  0.3314,  0.1020, -0.6083,  0.0356,
          0.0519, -1.0433],
        [-0.7799, -0.8931, -0.1844, -0.3242,  0.6701,  0.6911, -0.3257,  0.5089,
         -1.4004, -1.3998],
        [-0.2649, -0.3818, -0.3260, -0.7072, -0.0095,  0.1530, -0.0402,  0.2732,
         -0.3428, -0.8333],
        [-0.6247, -0.4311, -0.7299, -1.0482, -0.2558, -0.7946,  0.3372,  0.4470,
         -0.1404, -1.7728],
        [-0.5415, -0.2916, -0.1690, -0.8819,  0.4859, -0.6334, -0.1586,  0.3190,
         -0.3892, -0.8728],
        [-0.3192, -0.4002, -0.2290, -0.7917, -0.3734,  0.2274,  0.0495,  0.8865,
         -1.1970, -0.7701],
        [-0.6334, -0.8738, -0.2606, -1.1241, -0.0150, -0.1902,  0.3649, -0.2246,
         -0.9731, -0.7445],
        [-0.2142, -